In [70]:
import pandas as pd
import numpy as np

In [72]:
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/data/churn_comp_refresh/sample_submission_v2.csv')
print("Reading Members")
mem  = pd.read_csv("../Data/members_v3.csv")

Reading Members


In [73]:
mem.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time'],
      dtype='object')

In [74]:
## Merging Train and test with members 
test  = test.merge(mem[['msno','registered_via']], on ='msno' , how = 'left')
train  = train.merge(mem[['msno','registered_via']], on ='msno' , how = 'left')
del mem
train.registered_via.fillna(-1)
test.registered_via.fillna(-1)

0          7.0
1          9.0
2          7.0
3         -1.0
4         13.0
5          4.0
6          3.0
7         -1.0
8          9.0
9          4.0
10         7.0
11        -1.0
12         9.0
13         7.0
14         7.0
15         7.0
16         3.0
17         7.0
18        -1.0
19         7.0
20         7.0
21         3.0
22         7.0
23         9.0
24         7.0
25         7.0
26         7.0
27         4.0
28         7.0
29         7.0
          ... 
907441     7.0
907442     9.0
907443     7.0
907444     7.0
907445     7.0
907446     9.0
907447     9.0
907448     7.0
907449    -1.0
907450     7.0
907451     9.0
907452     7.0
907453     9.0
907454    -1.0
907455     4.0
907456     9.0
907457     7.0
907458     7.0
907459     4.0
907460     9.0
907461     9.0
907462     7.0
907463     7.0
907464     9.0
907465    -1.0
907466     7.0
907467     7.0
907468     7.0
907469     7.0
907470     7.0
Name: registered_via, Length: 907471, dtype: float64

In [76]:
# Logs info
print("Reading Logs File")
user_logs = pd.read_csv("../Data/user_logs.csv" , nrows = 1000000)
print("User Logs Info")
user_logs.info()
print(user_logs.head())

Reading Logs File
User Logs Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
msno          1000000 non-null object
date          1000000 non-null int64
num_25        1000000 non-null int64
num_50        1000000 non-null int64
num_75        1000000 non-null int64
num_985       1000000 non-null int64
num_100       1000000 non-null int64
num_unq       1000000 non-null int64
total_secs    1000000 non-null float64
dtypes: float64(1), int64(7), object(1)
memory usage: 68.7+ MB
                                           msno      date  num_25  num_50  \
0  rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150513       0       0   
1  rxIP2f2aN0rYNp+toI0Obt/N/FYQX8hcO1fTmmy2h34=  20150709       9       1   
2  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150105       3       3   
3  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150306       1       0   
4  yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=  20150501       3       

In [77]:
#Split the log data in train/test
train_logs = user_logs.loc[(user_logs.date>=20170101) & (user_logs.date<=20170130) ]
test_logs  = user_logs.loc[(user_logs.date>=20170201) & (user_logs.date<=20170228) ]

In [78]:
# We are calculating Median of total secs spend by each user on the platform , we are creating
# a column called total mins which is just sec times in minutes
md_train  = train_logs[['msno' , 'total_secs']].groupby('msno' , as_index = False).median()
md_train['total_minutes'] = ((md_train.total_secs)/60).astype('int') 
md_train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 44169 entries, 0 to 44168
Data columns (total 3 columns):
msno             44169 non-null object
total_secs       44169 non-null float64
total_minutes    44169 non-null int32
dtypes: float64(1), int32(1), object(1)
memory usage: 1.2+ MB


In [79]:
md_train.head()

,msno,total_secs,total_minutes
0,++BW1PJYU5OZi3n3+IDLiU+d1IL1VE/GLx6p64TDs6U=,5440.538,90
1,++N9C3T2DFC2uRZ9LYOJW0KSq1sG++54neJYETDmkkI=,8470.783,141
2,++Re/XY4v6500lA5vyrUA/WPsAfXqv1A/4d0FCbvpSI=,14516.856,241
3,++UGC5bVrIbJWXS5Q4B6Xxoj/yUsduvLdPSZx4tqcGk=,23229.442,387
4,++dv8FkN8F0u04Jl5glpj3gzD2ytDpOxAoc9LMsvF4E=,1902.010,31


In [80]:
# We are calculating Median of total secs spend by each user on the platform , we are creating
# a column called total mins which is just sec times in minutes
md_test  = test_logs[['msno' , 'total_secs']].groupby('msno' , as_index = False).median()
md_test['total_minutes'] = ((md_test.total_secs)/60).astype('int') 
md_test.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40791 entries, 0 to 40790
Data columns (total 3 columns):
msno             40791 non-null object
total_secs       40791 non-null float64
total_minutes    40791 non-null int32
dtypes: float64(1), int32(1), object(1)
memory usage: 1.1+ MB


In [81]:
md_test.head()

,msno,total_secs,total_minutes
0,++95tJZADNg8U8HKbYdxbbXIRsO6pw1zBK4tHI7BtZo=,1116.394,18
1,++BW1PJYU5OZi3n3+IDLiU+d1IL1VE/GLx6p64TDs6U=,8026.812,133
2,++IjleA6pVExACeYOj1kyyE1MNGt5vVTAX94T2VNasE=,3623.044,60
3,++U8PaO/jJnHP4wN/Vtg6RutbTJ4sqCRmdgI0xRjM2g=,4793.994,79
4,++jBVxyDBQXJ9ykE/NhDM5t/4Lo4v0IRxqNXMPLCFE0=,912.040,15


In [82]:
train.head()

,msno,is_churn,registered_via
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,9.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,9.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,9.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,9.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,9.0


In [83]:
## Merging all this columns with train.csv
train = train.merge(md_train , on = 'msno' , how = 'left')

In [84]:
train.isna().sum()

msno                   0
is_churn               0
registered_via    115770
total_secs        959440
total_minutes     959440
dtype: int64

In [85]:
train.shape

(992931, 5)

In [86]:
train.registered_via.fillna(-1 , inplace = True)
test.registered_via.fillna(-1 , inplace = True)

In [87]:
train.total_secs.fillna(train.total_secs.median() , inplace = True)
train.total_minutes.fillna(train.total_minutes.median() , inplace = True)

In [88]:
train.isna().sum()

msno              0
is_churn          0
registered_via    0
total_secs        0
total_minutes     0
dtype: int64

In [89]:
train.head()

,msno,is_churn,registered_via,total_secs,total_minutes
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,9.0,4642.477,77.0
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,9.0,4642.477,77.0
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,9.0,4642.477,77.0
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,9.0,4642.477,77.0
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,9.0,4642.477,77.0


In [90]:
test = test.merge(md_test , on = 'msno' , how = 'left')
test.total_secs.fillna(test.total_secs.median() , inplace = True)
test.total_minutes.fillna(test.total_minutes.median() , inplace = True)

In [91]:
test.head()

,msno,is_churn,registered_via,total_secs,total_minutes
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,7.0,4440.477,74.0
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,9.0,4440.477,74.0
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,7.0,4440.477,74.0
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,-1.0,4440.477,74.0
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,13.0,4440.477,74.0


In [92]:
print("churn mean")
#total churn mean (for exceptions)
base_mean = train.is_churn.mean()


churn mean


In [93]:
base_mean

0.06392287077349786

In [94]:
## We are grouping churn mean column by registered via and total_minutes and we are caclulating mean of is_churn
churn_mean = train[['registered_via', 'total_minutes','is_churn']].groupby(['registered_via', 'total_minutes'], as_index=False).mean()
churn_mean.columns = ['registered_via', 'total_minutes','churn_mean']
print(churn_mean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 0 to 2792
Data columns (total 3 columns):
registered_via    2793 non-null float64
total_minutes     2793 non-null float64
churn_mean        2793 non-null float64
dtypes: float64(3)
memory usage: 87.3 KB
None


In [95]:
churn_mean.head()

,registered_via,total_minutes,churn_mean
0,-1.0,77.0,0.050230
1,-1.0,96.0,0.000000
2,-1.0,215.0,1.000000
3,3.0,0.0,0.052632
4,3.0,1.0,0.187500


In [96]:
# We are merging with test dataset on basis of registered via and totalmins and filling values of churn mean , for nan values we fill it with base_mean
test = test.merge(churn_mean, how='left' )
test.churn_mean = test.churn_mean.fillna(base_mean)
print(test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 907471 entries, 0 to 907470
Data columns (total 6 columns):
msno              907471 non-null object
is_churn          907471 non-null int64
registered_via    907471 non-null float64
total_secs        907471 non-null float64
total_minutes     907471 non-null float64
churn_mean        907471 non-null float64
dtypes: float64(4), int64(1), object(1)
memory usage: 48.5+ MB
None


In [97]:
test.head()

,msno,is_churn,registered_via,total_secs,total_minutes,churn_mean
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0,7.0,4440.477,74.0,0.016393
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0,9.0,4440.477,74.0,0.057692
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0,7.0,4440.477,74.0,0.016393
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0,-1.0,4440.477,74.0,0.063923
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0,13.0,4440.477,74.0,0.000000


In [98]:
# replacing labels with actual values
test['is_churn'] = test.churn_mean


In [99]:
test[['msno','is_churn']].to_csv('subm.csv', index=False)


In [100]:
df_sub = pd.read_csv("subm.csv")

In [101]:
df_sub.head()

,msno,is_churn
0,4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=,0.016393
1,aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=,0.057692
2,rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=,0.016393
3,WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=,0.063923
4,aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=,0.000000


In [102]:
df_sub.shape

(907471, 2)

### Conclusions :
1. Log-loss 0.1590 
2. Top 63 % on Private Lb
3. Needs Lot of Improvements as it was based on Lot of Nan Values
4. Also hardly 2 features used so very biased model

### 3rd September Plan.
1. Try better feature Engineering Approaches using members_v3.csv and transactions_v3.csv , also Handle Nan values using some imputation -- See More Nice Kernels on Feature Engineering
2. Also try running Xgboost with better Params and 5 fold - 10 fold cv
3. Try to improve Log - Loss score using Xgboost
4. Try some linear models at end
